<a href="https://colab.research.google.com/github/1900690/kyouyu/blob/main/pytorch%E3%81%A7django.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DjangoアプリでPyTorchを使う方法
>"Djangoと事前に学習したPyTorch DenseNetモデルを使ってシンプルな画像分類のWebアプリを構築する"


このブログ記事では、事前に学習した[PyTorchのDenseNet 121モデル](https://pytorch.org/hub/pytorch_vision_densenet/)を使って簡単な画像分類アプリを構築しています。この画像分類モデルをHeroku上のDjango Webアプリ内で使えるようにしています。

<img src="https://github.com/stefanbschneider/blog/blob/master/_notebooks/pytorch-django/demo.gif?raw=1" alt="image classification demo" width="30%" />

これは、[PyTorch guide on deployment with Flask](https://pytorch.org/tutorials/intermediate/flask_rest_api_tutorial.html)の記事を参考にしています。ここでは、軽量ではありませんが、Flask よりも多くの機能を内蔵している Django を使った代替手段を紹介します。Django と Flask の違いについては、[このサイト](https://hackr.io/blog/flask-vs-django) を参照してください。

* [作成した画像分類アプリはこちら](https://pytorch-django.herokuapp.com/)
* [最終的なコードを含むGitHubリポジトリ](https://github.com/stefanbschneider/pytorch-django)
* 関連するブログ記事
    * [PyTorchを使い始める](https://stefanbschneider.github.io/blog/pytorch-getting-started)
    * [Heroku上でDjangoアプリを構築してデプロイする](https://stefanbschneider.github.io/blog/django-heroku)
    * [DjangoアプリにGoogle Analyticsを追加する](https://stefanbschneider.github.io/blog/django-google-analytics)

なお、デプロイしたアプリのロードには数秒かかることがあります。これは、Herokuの無料アプリコンテナを使用しているためで、未使用時には自動的に切断されます。

##初期セットアップ：Django と PyTorch をインストールする
必要なもの Python 3、GitHubとHerokuのアカウント。

In [33]:
!pip -q install torch        # ライブラリ「PyTorch」をインストール
!pip -q install torchvision  # 画像／ビデオ処理のPyTorch用追加パッケージもインストール
!pip -q install django torchvision  #Djangoをインストール

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pyparsing.py", line 589, in __init__
    self[name] = toklist[0]
TypeError: 'int' object is not subscriptable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(s

In [34]:
#インポートしてバージョンチェック
import torch
import torchvision
import django
print('PyTorch', torch.__version__)
print('Torchvision', torchvision.__version__)
print('django', django.__version__)

PyTorch 1.11.0+cu113
Torchvision 0.12.0+cu113
django 3.2.13


Django プロジェクト `pytorch_django` とアプリ `image_classification` を作成します。

In [ ]:
!django-admin startproject pytorch_django #pytorch_djangoディレクトリを作成、詳しくはhttps://docs.djangoproject.com/ja/4.0/intro/tutorial01/
%cd /content/pytorch_django #ディレクトリの中に移動
!python manage.py startapp image_classification #image_classificationアプリケーションを作成

以下のコードで`settings.py` 内に、 `'image_classification.apps.ImageClassificationConfig'` を `INSTALLED_APPS` リストに追加する。

In [48]:
##settings.pyを下記のコードで上書き、できるまで時間が借る
%%writefile settings.py
"""
pytorch_django プロジェクト用の Django 設定です。
Django 3.2.13 を使って 'django-admin startproject' で生成されます。
このファイルに関する詳細な情報は
https://docs.djangoproject.com/en/3.2/topics/settings/
設定の完全なリストとその値については、以下を参照してください。
https://docs.djangoproject.com/en/3.2/ref/settings/
"""

from pathlib import Path

# プロジェクト内のビルドパスはこのようにします。BASE_DIR / 'subdir'です。
BASE_DIR = Path(__file__).resolve().parent.parent


# クイックスタートの開発設定 - 製品版には不向き
# https://docs.djangoproject.com/en/3.2/howto/deployment/checklist/

# セキュリティ警告：プロダクションで使用する秘密鍵は秘密にしてください。
SECRET_KEY = 'django-insecure-138#a%l+%vl*^%lf$ir^$7u%!cgra-k97i)p@6j5&7y)w85bjh'

# セキュリティ警告：実運用環境ではデバッグをオンにした状態で実行しないでください。
DEBUG = True

ALLOWED_HOSTS = []


# アプリケーションの定義

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'image_classification.apps.ImageClassificationConfig',#ここを追加した
]

MIDDLEWARE = [
    'django.middleware.security.SecurityMiddleware',
    'django.contrib.sessions.middleware.SessionMiddleware',
    'django.middleware.common.CommonMiddleware',
    'django.middleware.csrf.CsrfViewMiddleware',
    'django.contrib.auth.middleware.AuthenticationMiddleware',
    'django.contrib.messages.middleware.MessageMiddleware',
    'django.middleware.clickjacking.XFrameOptionsMiddleware',
]

ROOT_URLCONF = 'pytorch_django.urls'

TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [],
        'APP_DIRS': True,
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ],
        },
    },
]

WSGI_APPLICATION = 'pytorch_django.wsgi.application'


# データベースについて
# https://docs.djangoproject.com/en/3.2/ref/settings/#databases

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': BASE_DIR / 'db.sqlite3',
    }
}


# パスワードの有効化について
# https://docs.djangoproject.com/en/3.2/ref/settings/#auth-password-validators

AUTH_PASSWORD_VALIDATORS = [
    {
        'NAME': 'django.contrib.auth.password_validation.UserAttributeSimilarityValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.MinimumLengthValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.CommonPasswordValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.NumericPasswordValidator',
    },
]


# 年齢制限・ローカライズについて
# https://docs.djangoproject.com/en/3.2/topics/i18n/

LANGUAGE_CODE = 'en-us'

TIME_ZONE = 'UTC'

USE_I18N = True

USE_L10N = True

USE_TZ = True


# 静的ファイル（CSS、JavaScript、画像)の管理について
# https://docs.djangoproject.com/en/3.2/howto/static-files/

STATIC_URL = '/static/'

# デフォルトの主なキーフィールドのタイプ
# https://docs.djangoproject.com/en/3.2/ref/settings/#default-auto-field

DEFAULT_AUTO_FIELD = 'django.db.models.BigAutoField'


Writing settings.py


In [49]:
#settings.pyファイルを表示
from google.colab import files
files.view("/content/pytorch_django/pytorch_django/settings.py")

<IPython.core.display.Javascript object>

まだエラーがないことを確認するために、Django の開発サーバを起動します。

In [50]:
!python manage.py runserver

Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).

You have 18 unapplied migration(s). Your project may not work properly until you apply the migrations for app(s): admin, auth, contenttypes, sessions.
Run 'python manage.py migrate' to apply them.
May 24, 2022 - 05:21:07
Django version 3.2.13, using settings 'pytorch_django.settings'
Starting development server at http://127.0.0.1:8000/
Quit the server with CONTROL-C.



```
python manage.py runserver
```

Go to `localhost:8000`:

![](https://github.com/stefanbschneider/blog/blob/master/_notebooks/pytorch-django/django-setup-complete.png?raw=1)



## PyTorch Image Classification

To classify uploaded images, I use a [DenseNet neural network](https://pytorch.org/hub/pytorch_vision_densenet/) that is pretrained on the
[ImageNet dataset](http://www.image-net.org/).
Since the web app is very simple and does not have any other functionality, I simply implement the image classification
inside the Django `image_classification/views.py` module.

> Note: This code is taken from a [PyTorch tutorial](https://pytorch.org/tutorials/intermediate/flask_rest_api_tutorial.html) and is under MIT license.


First, I load the pretrained DenseNet, switch to evaluation/inference mode (since I do not need any further training),
and load the mapping for predicted indices to human-readable labels for ImageNet.
The JSON-file containing the mapping is available [here](https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json)
and should be saved in the Django `static` directory under as defined in `STATICFILES_DIRS` (`settings.py`).

```python
import io
import os
import json

from torchvision import models
from torchvision import transforms
from PIL import Image
from django.conf import settings


# load pretrained DenseNet and go straight to evaluation mode for inference
# load as global variable here, to avoid expensive reloads with each request
model = models.densenet121(pretrained=True)
model.eval()

# load mapping of ImageNet index to human-readable label (from staticfiles directory)
# run "python manage.py collectstatic" to ensure all static files are copied to the STATICFILES_DIRS
json_path = os.path.join(settings.STATIC_ROOT, "imagenet_class_index.json")
imagenet_mapping = json.load(open(json_path))
```

> Important: It is important to load the pretrained model once as global variable and not inside the view function, which would reload the model on each request (expensive and slow!).


> Tip: Loading the static JSON file via `settings.STATIC_ROOT` should work both in development and production deployment but requires running `python manage.py collectstatic` first.


Then, I need a function to transform an uploaded image (passed in bytes) into the required format for DenseNet,
which is a 224 x 224 image with 3 RGB channels. The following code does this transformation and also normalizes the
image, returning the corresponding tensor:

```python
def transform_image(image_bytes):
    """
    Transform image into required DenseNet format: 224x224 with 3 RGB channels and normalized.
    Return the corresponding tensor.
    """
    my_transforms = transforms.Compose([transforms.Resize(255),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transforms(image).unsqueeze(0)
```

Finally, this function can be used inside the prediction function, where the transformed tensor of the uploaded image
is passed through the pretrained DenseNet model in a forward pass. Since I only do inference here not training,
I do not need a backward pass for backpropagation.
The model predicts the index of the corresponding ImageNet class, which is just an integer. To display a more useful
label, I retrieve the corresponding human-readable label from the `imagenet_mapping` dict that I created at the
beginning from the downloaded JSON file:

```python
def get_prediction(image_bytes):
    """For given image bytes, predict the label using the pretrained DenseNet"""
    tensor = transform_image(image_bytes)
    outputs = model.forward(tensor)
    _, y_hat = outputs.max(1)
    predicted_idx = str(y_hat.item())
    class_name, human_label = imagenet_mapping[predicted_idx]
    return human_label
```


## Django URL setup

Having the PyTorch classification logic implemented in `image_classification/views.py`, I now need to integrate it into
the Django app and really use it in a Django view and template.
For that, I first make some adjustments in the URLs by creating a separate `image_classification/urls.py` for the URLs
of the image classification app:
```python
from django.urls import path, include
from django.conf import settings
from django.conf.urls.static import static
from . import views

app_name = 'image_classification'
urlpatterns = [
    # two paths: with or without given image
    path('', views.index, name='index'),
] + static(settings.MEDIA_URL, document_root=settings.MEDIA_ROOT)
```
When visiting the main page of the web app, the requests are now directed to an `index` view, which I need to implement
next and which will make use of the previous PyTorch classification logic.
Before, I still need link these URLs to the project's URLs in `pytorch_django/urls.py` such that they become effective:
```python
urlpatterns = [
    path('', include('image_classification.urls')),
    path('admin/', admin.site.urls),
]
```


## Django Image Upload, Classification, and Display

Now, I implement the `index` view, which accepts an uploaded image, processes it, and passes it to the PyTorch
classification logic implemented above.
I also need a simple Django template to render the web interface, where users can upload an image and submit it for classification.
After classification, the template needs to show the predicted label.

### Form

For submitting uploaded images, I use a very simply Django form with an `ImageField` in `image_classification/forms.py`:
```python
from django import forms

class ImageUploadForm(forms.Form):
    image = forms.ImageField()
```

### View

I use this form inside my `index` view to accept uploaded images.
(`index` is how I called it in my `image_classification/urls.py` but it could be any
other name.)
Here, I just want to display the uploaded image and pass it to the PyTorch model for classification.
I do not want to (even temporarily) store it to the file system/disk.
Hence, inside the view (`image_classification/views.py`), I get the image from the form, get its byte representation (for PyTorch)
and create an image URI for displaying the image in the template later (see [StackOverflow](https://stackoverflow.com/a/40568024/2745116)):

```python
import base64
from django.shortcuts import render
from .forms import ImageUploadForm

def index(request):
    image_uri = None
    predicted_label = None

    if request.method == 'POST':
        # in case of POST: get the uploaded image from the form and process it
        form = ImageUploadForm(request.POST, request.FILES)
        if form.is_valid():
            # retrieve the uploaded image and convert it to bytes (for PyTorch)
            image = form.cleaned_data['image']
            image_bytes = image.file.read()
            # convert and pass the image as base64 string to avoid storing it to DB or filesystem
            encoded_img = base64.b64encode(image_bytes).decode('ascii')
            image_uri = 'data:%s;base64,%s' % ('image/jpeg', encoded_img)

            # get predicted label with previously implemented PyTorch function
            try:
                predicted_label = get_prediction(image_bytes)
            except RuntimeError as re:
                print(re)

    else:
        # in case of GET: simply show the empty form for uploading images
        form = ImageUploadForm()

    # pass the form, image URI, and predicted label to the template to be rendered
    context = {
        'form': form,
        'image_uri': image_uri,
        'predicted_label': predicted_label,
    }
    return render(request, 'image_classification/index.html', context)
```

### Template

The `index` view above calls Django's `render` function on a template `image_classification/index.html`,
which I need to create now (inside the `image_classification/templates` directory).
The template needs to show the form for uploading images and, after submitting and image, the uploaded image and its
predicted label.

{% raw %}
```html
<h1>Image Classification App</h1>
<p>A simple Django web app with a pretrained PyTorch DenseNet model will try to classify the selected image according to ImageNet labels. Uploaded images are not saved.</p>
<p><small>Further information:
    <a href="" target="_blank">Blog Post</a>,
    <a href="https://github.com/stefanbschneider/pytorch-django" target="_blank">GitHub</a></small>
</p>

<form method="post" enctype="multipart/form-data" style="margin-top: 50px; margin-bottom: 30px;">
    {% csrf_token %}
    {{ form }}
    <button type="submit" id="btnUpload" class="btn btn-primary">Upload</button>
</form>

{% if image_uri is not None %}
    {% if predicted_label is not None %}
        <div class="alert alert-primary" role="alert">
            Predicted label: <b>{{ predicted_label }}</b>
        </div>
    {% else %}
        <div class="alert alert-danger" role="alert">
            Prediction error. No label predicted.
        </div>
    {% endif %}

    <img src="{{ image_uri }}" class="img-fluid" alt="Uploaded image"
         style="max-width: min(500px, 100%); height: auto; margin-top: 30px;">
{% endif %}
```
{% endraw %}


The uploaded image uses the saved and passed image URI from before and does not save or load any image from disk,
which is important for privacy.

This template relies on some Bootstrap styling (see [my corresponding blog post](https://stefanbschneider.github.io/blog/django-bootstrap)),
but it is of course possible to omit that.

### Testing the App Locally

Running the app locally should now work without errors and show a simple page with the image upload form:

![](https://github.com/stefanbschneider/blog/blob/master/_notebooks/pytorch-django/image-upload.png?raw=1)

After uploading an image, the app shows the image and its classification below:

![](https://github.com/stefanbschneider/blog/blob/master/_notebooks/pytorch-django/classification.png?raw=1)

Here, it correctly classifies the image as a (tiger) cat.



## Deployment on Heroku

For (production) deployment of this simple web app on Heroku, a few extra steps are necessary.
Also refer to [my dedicated blog post](https://stefanbschneider.github.io/blog/django-heroku) on this topic for details.

### File Structure

For some reason, the default directory structure always breaks my Heroku deployment.
It works, when removing the parent `pytorch_django` directory like this:
```
# original structure when generating the project and app
pytorch_django
    image_classification
        ...
    pytorch_django
        ...
    manage.py
README.md

# after removing the parent directory
image_classification
    ...
pytorch_django
    ...
manage.py
README.md
```

### Setup and Production Settings

After creating the app on Heroku and enabling automatic deploys from the corresponding GitHub repo,
set the following config variables (in Heroku: Settings > Config Vars):
```
DJANGO_SETTINGS_MODULE: pytorch_django.prod_settings
DJANGO_SECRET_KEY: <randomly-generated-secret-key>
```
This indicates that Heroku should use a separate `prod_settings.py` rather than the `settings.py` used for development.
This `prod_settings.py` simply overwrites and disables debug mode, sets the production secret key, and allowed hosts.
It also makes use of the `django_heroku` package for further settings.
```python
import django_heroku
# default: use settings from main settings.py if not overwritten
from .settings import *

DEBUG = False
SECRET_KEY = os.getenv('DJANGO_SECRET_KEY', SECRET_KEY)
# adjust this to the URL of your Heroku app
ALLOWED_HOSTS = ['pytorch-django.herokuapp.com']
# Activate Django-Heroku.
django_heroku.settings(locals())
```

### Procfile and Requirements

Also, add a `Procfile` in the project root that indicates how to prepare the release and deployment on Heroku using `gunicorn`:
```
release: python manage.py migrate --no-input
web: gunicorn pytorch_django.wsgi
```
The paths depend on the project name and directory structure.

Also specify the requirements that need to be installed for the app in `requirements.txt`:
```
-f https://download.pytorch.org/whl/torch_stable.html
django==3.2
whitenoise==5.2.0
gunicorn==20.0.4
django-heroku==0.3.1
# cpu version of torch and torchvision for heroku to reduce slug size
torch==1.8.1+cpu
torchvision==0.9.1+cpu
```
For deployment on Heroku, it's important to use the CPU version of PyTorch since the slug size is otherwise too large
(above 500 MB), which leads to a build error (see [StackOverflow](https://stackoverflow.com/a/59122860/2745116)).
The free Herku dynos only support CPU anyways.


### Static Files

For serving static files (here, the JSON containing the ImageNet label mapping), configure `STATIC_ROOT`, `STATIC_URL`,
and `STATICFILES_DIR` in `settings.py`:
```python
STATIC_URL = '/static/'
# path to where static files are copied for deployment (eg, for heroku)
STATIC_ROOT = os.path.join(BASE_DIR, 'staticfiles')
# location of static files in local development: https://learndjango.com/tutorials/django-favicon-tutorial
STATICFILES_DIRS = [os.path.join(BASE_DIR, 'static'),]
```
For production, use `whitenoise` as described [here](https://devcenter.heroku.com/articles/django-assets#whitenoise).
Make sure to add the `staticfiles` directory to GitHub as it will not be created automatically by Django.

The `STATIC_ROOT` is used inside `views.py` (see above) to load the JSON file for mapping.
To copy all static files from their `STATICFILES_DIRS` to `STATIC_ROOT`, run
```
python manage.py collectstatic
```
This is only required once locally.
Heroku executes it on each deploy automatically.

### Testing the Deployed App

Check the Heroku activity/logs to see if the build and deployment are successful.
After successful deployment, access the app at its URL.
Mine is at [https://pytorch-django.herokuapp.com/](https://pytorch-django.herokuapp.com/).



## What Next?

Outcomes of this blog post:

* [Deployed app](https://pytorch-django.herokuapp.com/)
* [GitHub code](https://github.com/stefanbschneider/pytorch-django)

Other blog posts:

* [Posts related to Django](https://stefanbschneider.github.io/blog/categories/#django)
* [Posts related to PyTorch](https://stefanbschneider.github.io/blog/categories/#pytorch)
* [Posts related to Heroku](https://stefanbschneider.github.io/blog/categories/#heroku)

External links:

* [PyTorch tutorial on image classification with DenseNet and Flask](https://pytorch.org/tutorials/intermediate/flask_rest_api_tutorial.html)
* [PyTorch DenseNet information](https://pytorch.org/hub/pytorch_vision_densenet/)
* [ImageNet dataset](http://www.image-net.org/)